# Disclaimer

이 커널은 위키북스 출판 '파이썬 머신러닝 완벽 가이드' 도서를 다량 참고하여 작성하였습니다.

# 이 커널에서 얻게 될 것

- datetime형 칼럼 전처리
- 데이터의 분포와 np.log1p(), np.expm1()
- 카테고리형 변수 원-핫 인코딩
- train 데이터와 test 데이터의 칼럼 align
- 회귀 계수를 보고 결론 내리기

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 데이터 로드

train.csv 데이터 세트를 이용해 모델을 학습한 후 대여 횟수(count)를 예측해 보겠습니다.

In [ ]:
train_df = pd.read_csv('../input/bike-sharing-demand/train.csv')
test_df = pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

해당 데이터 세트에는 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재돼 있습니다. 데이터 세트의 주요 칼럼은 다음과 같습니다. 이 중 결정 값은 맨 마지막 칼럼인 count로 '대여 횟수'를 의미합니다.

- datetime: hourly date + timestamp
- season: 1=봄, 2=여름, 3=가을, 4=겨울
- holiday: 1=주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday: 1=주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
  - 1=맑음, 약간 구름 낀 흐림
  - 2=안개, 안개 + 흐림
  - 3=가벼운 눈, 가벼운 비 + 천둥
  - 4=심한 눈/비, 천둥/번개
- temp: 온도(섭씨)
- atemp: 체감온도(섭씨)
- humidity: 상대습도
- windspeed: 풍속
- casual: 사전에 등록되지 않은 사용자가 대여한 횟수
- registered: 사전에 등록된 사용자가 대여한 횟수
- count: 대여 횟수

In [ ]:
train_df.info()

In [ ]:
test_df.info()

테스트 데이터 세트에는 casual, registered, count 변수가 빠진 상태입니다.

# 데이터 전처리

Null 데이터는 없으며, 대부분의 칼럼이 int 또는 float형인데, datetime 칼럼만 object 형입니다. Datetime 칼럼의 경우 년-월-일 시:분:초 문자 형식으로 되어 있으므로 이에 대한 가공이 필요합니다. datetime을 년, 월, 일, 그리고 시간과 같이 4개의 속성으로 분리하겠습니다. 판다스에서는 datetime과 같은 형태의 문자열을 년도, 월, 일, 시간, 분, 초로 편리하게 변환하려면 먼저 문자열을 'datetime' 타입으로 변경해야 합니다.

In [ ]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])

In [ ]:
train_df['year'] = train_df['datetime'].apply(lambda x: x.year)
train_df['month'] = train_df['datetime'].apply(lambda x: x.month)
train_df['day'] = train_df['datetime'].apply(lambda x: x.day)
train_df['hour'] = train_df['datetime'].apply(lambda x: x.hour)

test_df['year'] = test_df['datetime'].apply(lambda x: x.year)
test_df['month'] = test_df['datetime'].apply(lambda x: x.month)
test_df['day'] = test_df['datetime'].apply(lambda x: x.day)
test_df['hour'] = test_df['datetime'].apply(lambda x: x.hour)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

새롭게 year, month, day, hour 칼럼이 추가되었습니다. 이제 datetime 칼럼은 삭제하겠습니다. 

또한 casual 칼럼은 사전에 등록하지 않은 사용자의 자전거 대여 횟수이고, registered는 사전에 등록한 사용자의 대여 횟수이며, casual + registered = count이므로 casual과 registered가 따로 필요하지는 않습니다. 오히려 상관도가 높아 예측을 저해할 우려가 있으므로 이 두 칼럼도 삭제하겠습니다.

In [ ]:
train_df = train_df.drop(['datetime', 'casual', 'registered'], axis=1)
test_df = test_df.drop(['datetime'], axis=1)

# 모델 선정

다음으로 다양한 회귀 모델을 데이터 세트에 적용해 예측 성능을 측정해 보겠습니다. 캐글에서 요구한 성능 평가 방법은 RMSLE(Root Mean Square Log Error)입니다. 즉, 오류 값의 로그에 대한 RMSE입니다. 아쉽게도 사이킷런은 RMSLE를 제공하지 않아서 RMSLE를 수행하는 성능 평가 함수를 직접 만들어 보겠습니다.

In [ ]:
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

위의 rmsle() 함수를 만들 때 한 가지 주의해야 할 점이 있습니다. rmsle를 구할 때 넘파이의 log() 함수를 이용하거나 사이킷런의 mean_squared_log_error()를 이용할 수도 있지만 데이터 값의 크기에 따라 오버플로/언더플로 오류가 발생하기 쉽습니다. 따라서 log()보다는 log1p()를 이용하는데, log1p()의 경우는 1+log() 값으로 log 변환값에 1을 더하므로 이런 문제를 해결해 줍니다. 그리고 log1p()로 변환된 값은 다시 넘파이의 expm1() 함수로 쉽게 원래의 스케일로 복원될 수 있습니다.

## 선형 회귀 모델

이제 회귀 모델을 이용해 자전거 대여 횟수를 예측해 보겠습니다. 회귀 모델을 적용하기 전에 데이터 세트에 대해서 먼저 처리해야 할 사항이 있습니다. 결괏값이 정규 분포로 돼 있는지 확인하는 것과 카테고리형 회귀 모델의 경우 원-핫 인코딩으로 피처를 인코딩하는 것입니다.

회귀에서 큰 예측 오류가 발생할 경우 가장 먼저 살펴볼 것은 Target 값의 분포가 왜곡된 형태를 이루고 있는지 확인하는 것입니다. Target 값의 분포는 정규 분포 형태가 가장 좋습니다. 그렇지 않고 왜곡된 경우에는 회귀 예측 성능이 저하되는 경우가 발생하기 쉽습니다.

In [ ]:
sns.distplot(train_df['count'])

count 칼럼 값이 정규 분포가 아닌 0~200 사이에 왜곡돼 있는 것을 알 수 있습니다. 이렇게 왜곡된 값을 정규 분포 형태로 바꾸는 가장 일반적인 방법은 로그를 적용해 변환하는 것입니다. 여기서는 넘파이의 log1p()를 이용하겠습니다. 이렇게 변경된 Target 값을 기반으로 학습하고 예측한 값은 다시 expm1() 함수를 적용해 원래 scale 값으로 원상 복구하면 됩니다. log1p()를 적용한 count값의 분포를 확인하겠습니다.

In [ ]:
sns.distplot(np.log1p(train_df['count']))

원하는 정규 분포 형태는 아니지만 변환하기 전보다는 왜곡 정도가 많이 향상됐습니다. 이를 이용해 LinearRegression 모델을 학습한 후 평가를 수행해 보겠습니다.

In [ ]:
train_df['count'] = np.log1p(train_df['count'])

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['count'], axis=1), train_df['count'], test_size=0.3)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('RMSLE:', rmsle(y_test_exp, pred_exp))

각 피쳐의 회귀 계수 값을 시각화해 보겠습니다.

In [ ]:
coef = pd.Series(lr_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)
sns.barplot(x=coef_sort.values, y=coef_sort.index)

year 피쳐의 회귀 계수 값이 독보적으로 큰 값을 가지고 있습니다. year는 2011, 2012 두 개의 값으로 돼 있습니다. year에 따라서 자전거 대여 횟수가 크게 영향을 받는다는 것은 납득하기 어렵습니다. 여름, 가을과 같이 자전거를 타기 좋은 계절이나 낮 시간대 등의 다양한 요소를 제외하고 year의 회귀 계수가 이렇게 큰 이유는 무엇일까요? year 피쳐는 연도를 뜻하므로 카테고리형 피쳐지만, 숫자형 값으로 돼 있습니다. 더군다나 아주 큰 값인 2011, 2012로 돼 있습니다. 사이킷런은 카테고리를 위한 데이터 타입이 없으며, 모두 숫자로 변환해야 합니다. 하지만 이처럼 숫자형 카테고리 값을 선형 회귀에 사용할 경우 회귀 계수를 연산할 때 이 숫자형 값에 크게 영향을 받는 경우가 발생할 수 있습니다. 따라서 선형 회귀에서는 이러한 피처 인코딩에 원-핫 인코딩을 적용해 변환해야 합니다.

판다스의 get_dummies()를 이용해 이러한 year 칼럼을 비롯해 month, day, hour, holiday, workingday, season, weather 칼럼도 모두 원-핫 인코딩한 후에 다시 예측 성능을 확인해 보겠습니다.

In [ ]:
train_df = pd.get_dummies(train_df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])
test_df = pd.get_dummies(test_df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])

In [ ]:
print(train_df.shape)
print(test_df.shape)

train_df와 test_df의 shape를 맞춰주기 위해 align을 사용합니다. (https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding )

In [ ]:
train_df, test_df = train_df.align(test_df, join='left', axis=1)
test_df = test_df.drop(['count'], axis=1)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.drop(['count'], axis=1), train_df['count'], test_size=0.3)

In [ ]:
lr_reg = LinearRegression()
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('LinearRegression RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
ridge_reg = Ridge(alpha=10)
ridge_reg.fit(X_train, y_train)
pred = ridge_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('Ridge RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
lasso_reg = Lasso(alpha=0.01)
lasso_reg.fit(X_train, y_train)
pred = lasso_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('Lasso RMSLE:', rmsle(y_test_exp, pred_exp))

원-핫 인코딩을 적용하고 나서 선형 회귀의 예측 성능이 많이 향상됐습니다. 원-핫 인코딩된 데이터 세트에서 회귀 계수가 높은 피쳐를 다시 시각화하겠습니다. 원-핫 인코딩으로 피쳐가 늘어났으므로 회귀 계수 상위 25개 피쳐를 추출해 보겠습니다. 

In [ ]:
coef = pd.Series(lr_reg.coef_, index=X_train.columns)
coef_sort = coef.sort_values(ascending=False)[:25]
sns.barplot(x=coef_sort.values, y=coef_sort.index)

월, 주말/주중, 그리고 계절 등 상식선에서 자전거를 타는 데 필요한 피쳐의 회귀 계수가 높아졌습니다. 이처럼 선형 회귀 수행 시에는 피처를 어떻게 인코딩하는가가 성능에 큰 영향을 미칠 수 있습니다.

## 트리 기반 회귀

이번에는 회귀 트리를 이용해 회귀 예측을 수행하겠습니다. 앞에서 적용한 Target 값의 로그 변환된 값과 원-핫 인코딩된 피쳐 세트를 그대로 이용해 랜덤 포레스트, GBM, XGBoost, LightGBM을 순차적으로 성능 평가해 보겠습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=500)
rf_reg.fit(X_train, y_train)
pred = rf_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('RandomForestRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
gbm_reg = GradientBoostingRegressor(n_estimators=500)
gbm_reg.fit(X_train, y_train)
pred = gbm_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('GradientBoostingRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
xgb_reg = XGBRegressor(n_estimators=500)
xgb_reg.fit(X_train, y_train)
pred = xgb_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('XGBRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

In [ ]:
lgbm_reg = LGBMRegressor(n_estimators=500)
lgbm_reg.fit(X_train, y_train)
pred = lgbm_reg.predict(X_test)

y_test_exp = np.expm1(y_test)
pred_exp = np.expm1(pred)
print('LGBMRegressor RMSLE:', rmsle(y_test_exp, pred_exp))

앞의 선형 회귀 모델보다 회귀 예측 성능이 개선됐습니다. 하지만 이것이 회귀 트리가 선형 회귀보다 더 나은 성능을 가진다는 의미는 아닙니다. 데이터 세트의 유형에 따라 결과는 얼마든지 달라질 수 있습니다.

# 예측(Prediction), 제출(Submission)

In [ ]:
X_train = train_df.drop(['count'], axis=1)
y_train = train_df['count']
X_test = test_df

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
lgbm_reg = LGBMRegressor(n_estimators=500)
lgbm_reg.fit(X_train, y_train)
pred = lgbm_reg.predict(X_test)

pred_exp = np.expm1(pred)

In [ ]:
submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
submission

In [ ]:
submission.loc[:, 'count'] = pred_exp
submission

In [ ]:
submission.to_csv('submission.csv', index=False)